In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

url = "http://www.fanfooty.com.au/resource/player.php?type=all"
headers = "fanfooty_player_id,fanfooty_afl_player_id,fanfooty_first_name,fanfooty_surname,fanfooty_team,fanfooty_status,fanfooty_number,fanfooty_birth_date,fanfooty_height,fanfooty_weight,fanfooty_state_of_origin,fanfooty_recruited_from,fanfooty_games,fanfooty_goals"

res = requests.get(url)
split_rows_list = res.text.split('"\r\n')
f = open('inputs/2020_fanfooty_player_ids_historical.csv','w')
f.write('{}\n'.format(headers)) 
for row in split_rows_list:
    f.write('{}\n'.format(row)) 
f.close()

In [2]:
df_player_list = pd.read_csv('inputs/2020_fanfooty_player_ids_historical.csv', encoding = "ISO-8859-1")
df_team_names = pd.read_csv('inputs/all_sources_team_names.csv')
df_player_list['short_team'] = df_player_list['fanfooty_team'].map(dict(df_team_names[['fanfooty_team_name_mid', 'sc_team_name']].values))
df_player_list = df_player_list.dropna(subset=['fanfooty_afl_player_id'])
df_player_list = df_player_list[df_player_list['fanfooty_afl_player_id'] != 0]

remove_duplicate_players = [
    {'player_id': 271128, 'first_name': 'Danny', 'surname': 'Butcher', 'team': 'Port Adelaide'},
    {'player_id': 296214, 'first_name': 'Jake', 'surname': 'Barratt', 'team': 'GWS'},
    {'player_id': 991933, 'first_name': 'Jason', 'surname': 'Cantstandya', 'team': 'Richmond'},
    {'player_id': 992351, 'first_name': 'Bailey', 'surname': 'Williams', 'team': 'None'}
]

for player in remove_duplicate_players:
    indexNames = df_player_list[
        (df_player_list['fanfooty_afl_player_id'] == player['player_id']) & 
        (df_player_list['fanfooty_first_name'] == player['first_name']) &
        (df_player_list['fanfooty_surname'] == player['surname']) &
        (df_player_list['fanfooty_team'] == player['team'])
    ].index
    df_player_list.drop(indexNames , inplace=True)

df_player_list.to_csv('inputs/2020_fanfooty_player_ids_historical.csv')
df_player_list

,fanfooty_player_id,fanfooty_afl_player_id,fanfooty_first_name,fanfooty_surname,fanfooty_team,fanfooty_status,fanfooty_number,fanfooty_birth_date,fanfooty_height,fanfooty_weight,fanfooty_state_of_origin,fanfooty_recruited_from,fanfooty_games,fanfooty_goals,short_team
3,43031,294472.0,Rory,Atkins,Adelaide,senior,21.0,1994-07-12,186.0,85.0,VIC,Calder Cannons,97.0,45.0,ADE
6,1011,980001.0,Nathan,Bassett,Adelaide,NaN,8.0,1976-12-07,190.0,88.0,SA,Norwood,210.0,25.0,ADE
7,44021,295026.0,James,Battersby,Adelaide,NaN,43.0,1995-11-07,177.0,78.0,SA,Sturt,NaN,NaN,ADE
8,46114,280974.0,Jonathon,Beech,Adelaide,NaN,1.0,1990-11-09,187.0,85.0,SA,West Adelaide,3.0,1.0,ADE
9,623,990006.0,James,Begley,Adelaide,NaN,28.0,1980-07-22,0.0,0.0,NaN,NaN,61.0,10.0,ADE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5041,47084,993894.0,Lachlan,Tiziani,GWS,NaN,33.0,1997-03-13,189.0,79.0,NSW,Broken Hill,0.0,0.0,GWS
5042,42215,291928.0,Gerald,Ugle,GWS,NaN,32.0,1993-01-31,180.0,80.0,WA,Beverley,3.0,1.0,GWS
5043,39670,280109.0,Callan,Ward,GWS,senior,8.0,1990-04-10,187.0,84.0,VIC,Western Jets,216.0,107.0,GWS
5044,43001,294305.0,Lachie,Whitfield,GWS,senior,6.0,1994-07-18,187.0,81.0,VIC,Dandenong Stingrays,129.0,59.0,GWS


In [3]:
df_player_list['fanfooty_afl_player_id'].value_counts()

290815.0     1
200031.0     1
293581.0     1
281292.0     1
281291.0     1
            ..
296289.0     1
1005000.0    1
298336.0     1
270606.0     1
1003520.0    1
Name: fanfooty_afl_player_id, Length: 2352, dtype: int64